# Import libraries and define functions + Initial setup

In [1]:
## Import Libraries
import pandas as pd
import numpy as np
from matplotlib_venn import venn2, venn2_circles, venn2_unweighted
from matplotlib_venn import venn3, venn3_circles
from matplotlib import pyplot as plt
%matplotlib inline
import seaborn as sns
from wordcloud import WordCloud
import csv


## Display all rows of pandas dataframes
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)

In [2]:
'''
name: relative_transcript_abundance

purpose: calculate relative transcript abundance

input: a dataframe with a ref_gene_id column identifying the transcript gene of origin and a cov columns with 
the coverage for the transcripts.

output: the same dataframe with a relative abundance column added
'''



def relative_transcript_abundance(df):
    
    ## Group by gene_id and get total expression for each gene (not counting introns)
    df_sums = df[["gene_id", "total_CPM"]].groupby("gene_id").sum()
    df_sums["total_CPM_gene"] = df_sums["total_CPM"]
    df_sums.drop(columns="total_CPM", inplace=True)
    
    ## Merge dataframe with total gene level CPM with regular transcript level CPM dataframe
    merged_df = pd.merge(df, df_sums, how='inner', on="gene_id")
    
    ## Calculater relative percent abundance for each transcript within its gene
    merged_df["relative_abundance_percent"] = ((merged_df["total_CPM"]/merged_df["total_CPM_gene"]) * 100)
    
    ## Rename total_CPM for transcript column
    merged_df["total_CPM_transcript"] = merged_df["total_CPM"]
    merged_df.drop(columns="total_CPM", inplace=True)


    return merged_df

In [3]:
'''
function name: fix_column_names

purpose: Fixing the column names, making them smaller, informative, and consistent

input: The raw counts dataframe for either genes or transcripts 

output: Same dataframe with improved column names
'''

def fix_column_names(df, is_gene=False):
    
    ## Check if this is a gene counts object
    if is_gene:
        
        ## Get count column names and create list of new column names
        count_columns = df.columns.tolist()
        list_new_names = ["gene_id"]
        
        ## gene_id comes in as index for gene counts data, make it into the first column instead
        df["gene_id"] = df.index
        cols = list(df.columns)
        cols = [cols[-1]] + cols[:-1]
        df = df[cols]
        df.reset_index(inplace=True, drop=True)
    
    ## If it is a transcript dataset
    else:
        ## Set count columns and create list of new names
        count_columns = df.columns[2:].tolist()
        list_new_names = [ "transcript_id", "gene_id"]
    
    ## Fix names one by one and add to list of new names
    for col in count_columns:
        col = col.split("_mapped")[0] + "_counts"
        list_new_names.append(col)
    
    ## Rename columns
    df.columns = list_new_names
    
    return df 

In [4]:
'''
function name: parse_df_columns

purpose: parsing the last aggregate column of the gtf/gff3 into useful columns and cleaning non-relevant columns

input: dataframe containining "raw" gtf/gff

output: dataframe containing gtf with useful columns ["gene_id", "transcript_id", etc...]
'''

def parse_df_columns(df, is_ref=True, is_transcript=False, is_prot=False, delete_other=True):

    if is_ref:

        ## Get gene ids
        df["gene_id"] = df["other"].str.split('";', expand=True)[0].str.extract("([^ \"]*$)", expand=True)
        
        ## Get gene names
        df["gene_name"] = df["other"].str.split("gene_name \"", expand=True)[1].str.split('\";', expand=True)[0]
        
        ## Get get transcript biotype
        df["gene_biotype"] = df["other"].str.split('gene_biotype "', expand=True)[1].str.split('"', expand=True)[0]
        
        ## If is transcript get transcript id and transcript biotype
        if is_transcript:
            df["transcript_id"] = df["other"].str.split('transcript_id "', expand=True)[1].str.split('"', expand=True)[0]
            df["transcript_biotype"] = df["other"].str.split('transcript_biotype "', expand=True)[1].str.split('"', expand=True)[0]
            
            ## If is prot get protein_id
            if is_prot:
                df["protein_id"] = df["other"].str.split('protein_id "', expand=True)[1].str.split('"', expand=True)[0]
                df["ccds_id"] = df["other"].str.split('ccds_id "', expand=True)[1].str.split('"', expand=True)[0]
                df["exon_number"] = df["other"].str.split('exon_number "', expand=True)[1].str.split('"', expand=True)[0]

        ## Drop "other" column
        df.drop(columns=["other", "dot_1", "dot_2"], inplace=True)
        

    else:

        ## Get gene ids
        df["gene_id"] = df["other"].str.split('";', expand=True)[0].str.extract("([^ \"]*$)", expand=True)

        ## Get transcript ids
        df["transcript_id"] = df["other"].str.split('transcript_id "', expand=True)[1].str.split('"', expand=True)[0]

        ## Get exon number
        df["exon_number"] = df["other"].str.split('exon_number "', expand=True)[1].str.split('"', expand=True)[0]

        ## Drop "other" column
        if delete_other:
            df.drop(columns=["other", "dot_1", "dot_2"], inplace=True)

    for col in df.columns:
        df.loc[df[col].isnull(), col] = np.NaN
        

    return df

In [5]:
'''
function name: calculate_cpm

purpose: Calculate CPM for the each sample given

input: Counts dataset

output: Counts dataset with CPM columns as well
'''

def calculate_cpm(df, is_gene=False):

    ## Set count columns if dataframe is gene counts
    if is_gene:
        count_columns = df.columns[1:].tolist()
    
    ## Set count columns if dataframe is transcript counts
    else:
        count_columns = df.columns[2:].tolist()

    ## Loop through counts columns to calculate CPM and add to the dataframe
    for col in count_columns:
        
        df[col] = round(df[col], 2)
        cpm_name = col.replace("_counts", "_CPM")
        df[cpm_name] = round(((df[col]/(df[col].sum())) * 1000000), 2)
    
    return df  

In [6]:
## define ggplot colors
ggplot2_colors = ["#F8766D", "#CD9600", "#7CAE00", "#00BE67", "#00BFC4", "#00A9FF", "#C77CFF", "#FF61CC"]

In [7]:
## Open original reference
original_ref = pd.read_csv("../../references/Homo_sapiens.GRCh38.107_ERCC.gtf", header=None, delimiter="\t", low_memory=False, 
                       names=["chr", "source", "type", "start", "end", "dot_1", "strand", "dot_2", "other"], comment="#")

original_ref = original_ref.loc[~original_ref["chr"].str.startswith("ERCC-")]

## Parse through reference to get gene names and ids
orig_ref = original_ref.loc[original_ref["type"]=="gene"].copy()
orig_ref = parse_df_columns(orig_ref, is_ref=True)

## Import disease relevant genes
disease_relevant_genes = pd.read_csv("../../references/medically_relevant_genes_02-04-2023_UPDATED.tsv", sep="\t")

## Brain disease genes
brain_disease_gene_ids = pd.read_csv("../../references/brain_disease_genes_only_IDs.tsv", sep="\t")
brain_disease_annotations = pd.read_csv("../../references/brain_disease_genes_with_disease.tsv", sep="\t")

## Import AD Genes
ad_names = pd.read_csv("../../references/AD_gwas_genes.tsv", sep="\t")

## Create disease relevant list including chromosome
disease_relevant_genes_annotated = disease_relevant_genes.merge(orig_ref[["gene_id", "gene_name", "chr"]], 
                                                               how="inner", on=["gene_id", "gene_name"])

## Create list of protein coding genes
protein_coding_ref = orig_ref.loc[orig_ref["gene_biotype"] == "protein_coding"].copy()

## Import and parse through extended annotations
ref = pd.read_csv("../../data/raw/nextflow_pipeline_output/bambu_discovery/extended_annotations.gtf", header=None, delimiter="\t",
                        low_memory=False, comment="#", names=["chr", "source", "type", "start", "end", "dot_1", "strand", "dot_2", "other"])

ref = ref.loc[~ref["chr"].str.startswith("ERCC-")]

ref = parse_df_columns(ref, is_ref=False)

ref_transcripts = ref.loc[ref["type"] == "transcript"].copy()

# - Figure 2

In [8]:
## Import data for transcript level counts and fix column names

df = pd.read_csv("../../data/raw/nextflow_pipeline_output/bambu_discovery/counts_transcript.txt", 
                           delimiter="\t", low_memory=False, header=0)

df = fix_column_names(df, is_gene=False)

In [9]:
## Calculate total counts
df["total_counts"] = df[df.filter(regex='count').columns].sum(axis=1)

In [10]:
## Calculate CPM and drop count columns
df = calculate_cpm(df, is_gene=False)

df = df[df.columns.drop(list(df.filter(regex='counts')))].copy()

In [11]:
## Drop ERCCs
df = df.loc[~df["gene_id"].str.startswith("ERCC")].copy()

In [12]:
## Calculate median CPM
df["median_CPM"] = df[df.filter(regex='[0-9]_CPM').columns].median(axis=1)

In [13]:
## Annotate transcritps in counts matrix
df = df.merge(ref_transcripts[["gene_id", "transcript_id", "chr"]], on=["gene_id", "transcript_id"], how="inner")

In [14]:
## Create novel transcripts annotations
df_novel = df.loc[df["transcript_id"].str.startswith("BambuTx")].copy()
df_novel = df_novel.loc[df_novel["chr"] != "MT"].copy()
df_novel = df_novel[df_novel["median_CPM"] > 1].copy()


In [15]:
## Create novel mitochondrial transcript dataframe
df_novel_mito = df.loc[df["transcript_id"].isin(["BambuTx1845", "BambuTx1846", "BambuTx1847",
                                                 "BambuTx1848", "BambuTx1850"])].copy()

In [16]:
## Create final novel dataframe
df_novel = pd.concat([df_novel, df_novel_mito])

In [17]:
## Get novel ids
df_novel_ids = df_novel[["gene_id", "transcript_id"]].copy()

In [18]:
## Create known transcripts annotations
df_known = df.loc[~(df["transcript_id"].str.startswith("BambuTx"))].copy()

In [19]:
## Parse through reference to get gene names and ids
prot_ref = original_ref.loc[original_ref["type"]=="transcript"].copy()
prot_ref = parse_df_columns(prot_ref, is_ref=True, is_transcript=True)
prot_ref = prot_ref.loc[prot_ref["transcript_biotype"] == "protein_coding"].copy()

In [20]:
## Add all mapt transcripts
prot_ref_mapt = prot_ref.loc[prot_ref["gene_name"] == "MAPT"].copy()
prot_ref_mapt_ids = prot_ref_mapt[["gene_id", "transcript_id"]].copy()

df_novel_ids = pd.concat([df_novel_ids, prot_ref_mapt_ids])

In [21]:
## Filter known transcripts
df_known = df_known.loc[((df_known["chr"] == "MT") | (df_known["gene_id"].isin(df_novel_ids["gene_id"])))].copy()
df_known = df_known.loc[df_known["transcript_id"].isin(prot_ref["transcript_id"])].copy()
df_known_ids = df_known[["gene_id", "transcript_id"]].copy()

In [22]:
## Drop any duplicates
df_novel_ids.drop_duplicates(inplace=True)
df_known_ids.drop_duplicates(inplace=True)

In [23]:
## Get all ids for relevant transcripts
all_ids = pd.concat([df_known_ids, df_novel_ids])

In [24]:
## Import and parse through extended annotations
ref = pd.read_csv("../../data/raw/nextflow_pipeline_output/bambu_discovery/extended_annotations.gtf", header=None, delimiter="\t",
                        low_memory=False, comment="#", names=["chr", "source", "type", "start", "end", "dot_1", "strand", "dot_2", "other"])

ref = parse_df_columns(ref, is_ref=False, delete_other=False)

In [25]:
## Create final reference for mass spec from cell line
ref_new = ref.loc[ref["transcript_id"].isin(df_novel_ids["transcript_id"])].copy()

## Only keep relevant columns
ref_new = ref_new[['chr', 'source', 'type', 'start', 'end', 'dot_1', 'strand', 'dot_2', 'other']].copy()

## Output reference
ref_new.to_csv("../../data/processed/other/mass_spec_annotations/NEW_annotation_for_cell_line_mass_spec.gtf", 
                 index=False, header=False, sep="\t", quoting=csv.QUOTE_NONE)

In [26]:
## Create final reference for mass spec from cell line
ref_known = ref.loc[ref["transcript_id"].isin(df_known_ids["transcript_id"])].copy()

## Only keep relevant columns
ref_known = ref_known[['chr', 'source', 'type', 'start', 'end', 'dot_1', 'strand', 'dot_2', 'other']].copy()

## Output reference
ref_known.to_csv("../../data/processed/other/mass_spec_annotations/KNOWN_annotation_for_cell_line_mass_spec.gtf", 
                 index=False, header=False, sep="\t", quoting=csv.QUOTE_NONE)

In [27]:
## Create final reference for mass spec from cell line
ref_final = ref.loc[ref["transcript_id"].isin(all_ids["transcript_id"])].copy()

## Only keep relevant columns
ref_final = ref_final[['chr', 'source', 'type', 'start', 'end', 'dot_1', 'strand', 'dot_2', 'other']].copy()

## Output reference
ref_final.to_csv("../../data/processed/other/mass_spec_annotations/ALL_annotation_for_cell_line_mass_spec.gtf", 
                 index=False, header=False, sep="\t", quoting=csv.QUOTE_NONE)